# Algorithm Implement

In [72]:
import torch
import torch.nn as nn
import torch.nn.functional as F
time_period = 15
class Q_Network(nn.Module):
    '''
    The input of this network should have shape (num_frame, 80, 80)
    '''

    def __init__(self, num_frame, num_action):
        super(Q_Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=num_frame, out_channels=32, kernel_size=(2,1), stride=1, padding=2)  # 16, 20, 20
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(2,1), stride=1)  # 32, 9, 9
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(2,1), stride=1)  # 32, 9, 9
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(2,1), stride=1)  # 32, 9, 9
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(2,2), stride=1)  # 32, 9, 9
        self.pool = nn.AvgPool2d(kernel_size=(2,1))
        self.fc1 = nn.Linear(672, 256)
        self.fc2 = nn.Linear(256, num_action)
        self.fc3 = nn.Linear(256, 1)

    def forward(self, image):
        x = F.relu(self.pool(self.conv1(image)))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.relu(self.pool(self.conv3(x)))
        x = x.view(-1, 672)
        x = F.relu(self.fc1(x))
        x1 = self.fc2(x)
        x1 = x1 - torch.max(x1, dim=1, keepdim=True)[0]
        x2 = self.fc3(x)
        return x1 + x2

In [73]:
from torchsummary import summary

# Data Loading

In [74]:
pwd

'/content'

In [75]:
import pandas as pd
import pickle
import pandas as pd
import pickle
# file = open('/content/drive/MyDrive/TeamCapstone/stock_data_full.bin', 'rb')
# data = pickle.load(file)
# file.close()
# codes = ['AAPL','AMZN','C','GOOG','JPM','NFLX','PLTR']
# for i in range(len(codes)):
#     data[i]['symbol'] = codes[i]

df = pd.read_csv('/content/drive/MyDrive/TeamCapstone/concat_data.csv')
df=df[['date', 'open', 'high', 'low', 'close', 'volume',
       'positive', 'neutral', 'negative', 'tic']]
df['date'] = [x[:10] for x in df['date']]
df = df[(df['date']>='2022-01-01') & (df['date']<'2023-09-30')]


In [76]:
data = pd.read_csv('/content/drive/MyDrive/TeamCapstone/min_data_adjust.csv')

In [77]:
stock_data = data[data['symbol']=='AAPL']

In [78]:
stock_data

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AAPL,2022-01-03 09:00:00+00:00,176.23,176.23,176.1800,176.1800,1118.0,65.0,176.210000
1,AAPL,2022-01-03 09:02:00+00:00,176.30,176.31,176.2800,176.2800,1218.0,26.0,176.300000
2,AAPL,2022-01-03 09:03:00+00:00,176.25,176.27,176.2500,176.2700,814.0,30.0,176.260000
3,AAPL,2022-01-03 09:04:00+00:00,176.20,176.20,176.1200,176.1200,3744.0,114.0,176.180000
4,AAPL,2022-01-03 09:05:00+00:00,176.17,176.17,176.1700,176.1700,464.0,33.0,176.150000
...,...,...,...,...,...,...,...,...,...
343433,AAPL,2023-09-29 23:53:00+00:00,171.30,171.30,171.3000,171.3000,209.0,8.0,171.305766
343434,AAPL,2023-09-29 23:54:00+00:00,171.30,171.30,171.3000,171.3000,810.0,16.0,171.305889
343435,AAPL,2023-09-29 23:57:00+00:00,171.32,171.32,171.3200,171.3200,439.0,20.0,171.330957
343436,AAPL,2023-09-29 23:58:00+00:00,171.30,171.30,171.2699,171.2699,532.0,11.0,171.282998


In [79]:
stock_df = df[df['tic']=='AAPL']

In [80]:
stock_df

,date,open,high,low,close,volume,positive,neutral,negative,tic
17,2022-01-03,177.830002,182.880005,177.710007,182.009995,104487900,-2.525743,3.722111,-3.922445,AAPL
21,2022-01-04,182.630005,182.940002,179.119995,179.699997,99310400,-2.752612,3.370780,-3.351379,AAPL
26,2022-01-05,179.610001,180.169998,174.639999,174.919998,94537600,-2.561095,3.561730,-3.588621,AAPL
35,2022-01-06,172.699997,175.300003,171.639999,172.000000,96904000,-2.294448,3.207229,-3.612424,AAPL
42,2022-01-07,172.889999,174.139999,171.029999,172.169998,86709100,-2.325235,3.084295,-3.352122,AAPL
...,...,...,...,...,...,...,...,...,...,...
3028,2023-09-25,174.199997,176.970001,174.149994,176.080002,46172700,-2.361765,3.181928,-3.037302,AAPL
3034,2023-09-26,174.820007,175.199997,171.660004,171.960007,64588900,-1.893191,2.688069,-3.369864,AAPL
3045,2023-09-27,172.619995,173.039993,169.050003,170.429993,66921800,-3.139558,3.359877,-2.654129,AAPL
3049,2023-09-28,169.339996,172.029999,167.619995,170.690002,56294400,-2.045589,2.791628,-3.063628,AAPL


# Technical Indicators

In [81]:
!pip install finta

In [82]:
from finta import TA

In [83]:
stock_df['SMA42'] = TA.SMA(stock_df, 42)
stock_df['SMA5'] = TA.SMA(stock_df, 5)
stock_df['SMA15'] = TA.SMA(stock_df, 15)
stock_df['AO'] = TA.AO(stock_df)
stock_df['OVB'] = TA.OBV(stock_df)
stock_df[['VW_MACD','MACD_SIGNAL']] = TA.VW_MACD(stock_df)
stock_df['RSI'] = TA.RSI(stock_df)
stock_df['CMO'] = TA.CMO(stock_df)

In [84]:
stock_df = stock_df.dropna()

In [85]:
stock_df.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'positive', 'neutral',
       'negative', 'tic', 'SMA42', 'SMA5', 'SMA15', 'AO', 'OVB', 'VW_MACD',
       'MACD_SIGNAL', 'RSI', 'CMO'],
      dtype='object')

# Replay Buffer

In [86]:
class SumTree:

    def __init__(self, capacity):

        self.capacity = capacity
        # the first capacity-1 positions are not leaves
        self.vals = [0 for _ in range(2*capacity - 1)] # think about why if you are not familiar with this

    def retrive(self, num):
        '''
        This function find the first index whose cumsum is no smaller than num
        '''
        ind = 0 # search from root
        while ind < self.capacity-1: # not a leaf
            left = 2*ind + 1
            right = left + 1
            if num > self.vals[left]: # the sum of the whole left tree is not large enouth
                num -= self.vals[left] # think about why?
                ind = right
            else: # search in the left tree
                ind = left
        return ind - self.capacity + 1

    def update(self, delta, ind):
        '''
        Change the value at ind by delta, and update the tree
        Notice that this ind should be the index in real memory part, instead of the ind in self.vals
        '''
        ind += self.capacity - 1
        while True:
            self.vals[ind] += delta
            if ind == 0:
                break
            ind -= 1
            ind //= 2

In [87]:
# from collections import deque

# test = deque(maxlen=5)
# for i in range(10):
#     test.append(i)
#     print(test)

import numpy as np
import random
import bisect
import torch

ALPHA = 0.5
EPSILON = 0.05
TD_INIT = 1

class Replay_Buffer:
    '''
    Vanilla replay buffer
    '''

    def __init__(self, capacity=int(1e6), batch_size=None):

        self.capacity = capacity
        self.memory = [None for _ in range(capacity)] # save tuples (state, action, reward, next_state, done)
        self.ind_max = 0 # how many transitions have been stored

    def remember(self, state, action, reward, next_state, done):

        ind = self.ind_max % self.capacity
        self.memory[ind] = (state, action, reward, next_state, done)
        self.ind_max += 1

    def sample(self, k):
        '''
        return sampled transitions. Make sure that there are at least k transitions stored before calling this method
        '''
        index_set = random.sample(list(range(len(self))), k)
        states = torch.from_numpy(np.vstack([self.memory[ind][0] for ind in index_set])).float()
        actions = torch.from_numpy(np.vstack([self.memory[ind][1] for ind in index_set])).long()
        rewards = torch.from_numpy(np.vstack([self.memory[ind][2] for ind in index_set])).float()
        next_states = torch.from_numpy(np.vstack([self.memory[ind][3] for ind in index_set])).float()
        dones = torch.from_numpy(np.vstack([self.memory[ind][4] for ind in index_set]).astype(np.uint8)).float()

        return states, actions, rewards, next_states, dones

    def __len__(self):
        return min(self.ind_max, self.capacity)

class Rank_Replay_Buffer:
    '''
    Rank-based replay buffer
    '''

    def __init__(self, capacity=int(1e6), batch_size=64):
        self.capacity = capacity
        self.batch_size = batch_size
        self.alpha = ALPHA
        self.memory = [None for _ in range(capacity)]
        self.segments = [-1] + [None for _ in range(batch_size)] # the ith index will be in [segments[i-1]+1, segments[i]]

        self.errors = [] # saves (-TD_error, index of transition), sorted
        self.memory_to_rank = [None for _ in range(capacity)]

        self.ind_max = 0 # how many transitions have been stored
        self.total_weights = 0 # sum of p_i
        self.cumulated_weights = []

    def remember(self, state, action, reward, next_state, done):
        index = self.ind_max % self.capacity
        if self.ind_max >= self.capacity: # memory is full, need to pop
            self.pop(index)
        else: # memory is not full, need to adjust weights and find segment points
            self.total_weights += (1/(1+self.ind_max))**self.alpha # memory is not full, calculate new weights
            self.cumulated_weights.append(self.total_weights)
            self.update_segments()

        max_error = -self.errors[0][0] if self.errors else 0
        self.insert(max_error, index)
        self.memory[index] = (state, action, reward, next_state, done)
        self.ind_max += 1

    def sample(self, batch_size=None): # notive that batch_size is not used. It's just to unify the calling form
        index_set = [random.randint(self.segments[i]+1, self.segments[i+1]) for i in range(self.batch_size)]
        probs = torch.from_numpy(np.vstack([(1/(1+ind))**self.alpha/self.total_weights for ind in index_set])).float()

        index_set = [self.errors[ind][1] for ind in index_set]
        states = torch.from_numpy(np.vstack([self.memory[ind][0] for ind in index_set])).float()
        actions = torch.from_numpy(np.vstack([self.memory[ind][1] for ind in index_set])).long()
        rewards = torch.from_numpy(np.vstack([self.memory[ind][2] for ind in index_set])).float()
        next_states = torch.from_numpy(np.vstack([self.memory[ind][3] for ind in index_set])).float()
        dones = torch.from_numpy(np.vstack([self.memory[ind][4] for ind in index_set]).astype(np.uint8)).float()
        for ind in index_set:
            self.pop(ind)

        return index_set, states, actions, rewards, next_states, dones, probs

    def insert(self, error, index):
        '''
        Input :
            error : the TD-error of this transition
            index : the location of this transition
        insert error into self.errors, update self.memory_to_rank and self.rank_to_memory accordingly
        '''
        ind = bisect.bisect(self.errors, (-error, index))
        self.memory_to_rank[index] = ind
        self.errors.insert(ind, (-error, index))
        for i in range(ind+1, len(self.errors)):
            self.memory_to_rank[self.errors[i][1]] += 1

    def pop(self, index):
        '''
        Input :
            index : the location of a transition
        remove this transition, update self.memory_to_rank and self.rank_to_memory accordingly
        '''
        ind = self.memory_to_rank[index]
        self.memory_to_rank[index] = None
        self.errors.pop(ind)
        for i in range(ind, len(self.errors)):
            self.memory_to_rank[self.errors[i][1]] -= 1

    def update_segments(self):
        '''
        Update the segment points.
        '''
        if self.ind_max+1 < self.batch_size: # if there is no enough transitions
            return None
        for i in range(self.batch_size):
            ind = bisect.bisect_left(self.cumulated_weights, self.total_weights*((i+1)/self.batch_size))
            self.segments[i+1] = max(ind, self.segments[i]+1)

    def __len__(self):
        return min(self.capacity, self.ind_max)


class Proportion_Replay_Buffer:
    '''
    Proportion-based replay buffer
    '''

    def __init__(self, capacity=int(1e6), batch_size=None):
        self.capacity = capacity
        self.alpha = ALPHA
        self.memory = [None for _ in range(capacity)]
        self.weights = SumTree(self.capacity)
        self.default = TD_INIT
        self.ind_max = 0

    def remember(self, state, action, reward, next_state, done):
        index = self.ind_max % self.capacity
        self.memory[index] = (state, action, reward, next_state, done)
        delta = self.default+EPSILON - self.weights.vals[index+self.capacity-1]
        self.weights.update(delta, index)
        self.ind_max += 1

    def sample(self, batch_size):
        index_set = [self.weights.retrive(self.weights.vals[0]*random.random()) for _ in range(batch_size)]
        #print(index_set)
        probs = torch.from_numpy(np.vstack([self.weights.vals[ind+self.capacity-1]/self.weights.vals[0] for ind in index_set])).float()

        states = torch.from_numpy(np.vstack([self.memory[ind][0] for ind in index_set])).float()
        actions = torch.from_numpy(np.vstack([self.memory[ind][1] for ind in index_set])).long()
        rewards = torch.from_numpy(np.vstack([self.memory[ind][2] for ind in index_set])).float()
        next_states = torch.from_numpy(np.vstack([self.memory[ind][3] for ind in index_set])).float()
        dones = torch.from_numpy(np.vstack([self.memory[ind][4] for ind in index_set]).astype(np.uint8)).float()

        return index_set, states, actions, rewards, next_states, dones, probs

    def insert(self, error, index):
        delta = error+EPSILON - self.weights.vals[index+self.capacity-1]
        self.weights.update(delta, index)

    def __len__(self):
        return min(self.capacity, self.ind_max)

In [88]:
tst = None
import random
from collections import deque
import torch
import torch.optim as optim
import numpy as np

# from networks import *

class Agent:

    def __init__(self, state_size, action_size, bs, lr, tau, gamma, device, visual=False):
        '''
        When dealing with visual inputs, state_size should work as num_of_frame
        '''
        self.state_size = state_size
        self.action_size = action_size
        self.bs = bs
        self.lr = lr
        self.tau = tau
        self.gamma = gamma
        self.device = device
        self.Q_local = Q_Network(self.state_size, self.action_size).to(device)
        self.Q_target = Q_Network(self.state_size, self.action_size).to(device)
        self.soft_update(1)
        self.optimizer = optim.Adam(self.Q_local.parameters(), self.lr)
        self.memory = Proportion_Replay_Buffer(int(1e5), bs)
        self.tst = None

    def act(self, state, eps=0):
        if random.random() > eps:
            state = torch.tensor(state, dtype=torch.float32).to(self.device)
            with torch.no_grad():
                action_values = self.Q_local(state)
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self):
        index_set, states, actions, rewards, next_states, dones, probs = self.memory.sample(self.bs)
        w = 1/len(self.memory)/probs
        w = w/torch.max(w)
        w = w.to(self.device)
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        Q_values = self.Q_local(states)
        Q_values = torch.gather(input=Q_values, dim=-1, index=actions)
        with torch.no_grad():
            Q_targets = self.Q_target(next_states)
            Q_targets, _ = torch.max(input=Q_targets, dim=-1, keepdim=True)
            Q_targets = rewards + self.gamma * (1 - dones) * Q_targets

        deltas = Q_values - Q_targets
        loss = (w*deltas.pow(2)).mean()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        deltas = np.abs(deltas.detach().cpu().numpy().reshape(-1))
        for i in range(self.bs):
            self.memory.insert(deltas[i], index_set[i])
    def soft_update(self, tau):
        for target_param, local_param in zip(self.Q_target.parameters(), self.Q_local.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

In [89]:
indicators = ['open', 'high', 'low', 'close', 'volume', 'positive', 'neutral', 'negative','SMA42', 'SMA5', 'SMA15', 'AO', 'OVB','VW_MACD',
       'MACD_SIGNAL', 'RSI', 'CMO']

In [90]:
class Stock_Env:
    def __init__(self, initial_asset, data, cost):
        self.asset = initial_asset
        self.cash = initial_asset
        self.stock = 0
        self.data = data
        self.time = data.iloc[time_period]['date']
        self.cost = cost
        self.history=[]
        self.total_cost = 0
        self.initial_asset = initial_asset
        self.rowid = time_period
        self.action_space = np.array(list(range(11)))

    def reset(self):
        self.asset = self.initial_asset
        self.cash = self.initial_asset
        self.stock = 0
        self.time = self.data.iloc[100]['date']
        self.history=[]
        self.total_cost = 0
        self.rowid = time_period
        return self.data[:time_period][indicators].values

    def step(self, action):
        done = False
        states = self.data.iloc[self.rowid]
        self.rowid +=1
        if self.rowid == len(self.data)-1:
            done = True
        next_state = self.data.iloc[self.rowid]
        last_asset = self.asset
        price = next_state['open']
        old_asset = self.cash + self.stock*price
        self.asset = old_asset
        target_value = action*0.1*self.asset
        distance = target_value - self.stock*price
        stock_distance = int(distance/(price*(1+self.cost)))
        self.stock += stock_distance
        self.cash = self.cash - distance - np.abs(stock_distance*self.cost*price)
        self.asset = self.cash+self.stock*price
        market_value = self.stock * next_state['close']
        self.asset = market_value + self.cash
        reward = self.asset - last_asset
        self.time = next_state['date']
        # self.stock = stock
        return (self.data[self.rowid-time_period:self.rowid][indicators].values, reward, done)

In [91]:
#env = gym.make()
env = Stock_Env(1000000, stock_df, 0.002)
num_episode = 5
max_t = 1000
reward_log = []

for _ in range(num_episode):

    # initialize
    env.reset()
    t = 0
    episodic_reward = 0

    for t in range(max_t):

        #env.render()
        action = np.random.randint(11) # random action
        _, reward, done = env.step(action)
        episodic_reward += reward
        if done:
            break

    reward_log.append(episodic_reward)

In [92]:
agent = Agent(1, len(env.action_space), 64, 0.001, 0.001, 0.99, 'cuda',True)

In [93]:
import warnings
warnings.filterwarnings('ignore')
#env = gym.make()
num_episode = 20000
max_t = 1000
reward_log = []
average_log = [] # monitor training process
eps = 1
eps_decay = 0.995
eps_min = 0.01
C = 4 # update weights every C steps

def train(env, agent, num_episode, eps_init, eps_decay, eps_min, max_t, num_frame=1, constant=0):
    rewards_log = []
    average_log = []
    eps = eps_init

    for i in range(1, 1 + num_episode):

        episodic_reward = 0
        done = False
        frame = env.reset()
        state_deque = deque(maxlen=num_frame)
        for _ in range(num_frame):
            state_deque.append(frame)
        state = np.stack(state_deque, axis=0)
        state = np.expand_dims(state, axis=0)
        t = 0

        while not done and t < max_t:

            t += 1
            action = agent.act(state, eps)
            frame, reward, done = env.step(action)
            state_deque.append(frame)
            next_state = np.stack(state_deque, axis=0)
            next_state = np.expand_dims(next_state, axis=0)
            agent.memory.memory.append((state, action, reward, next_state, done))

            if t % 5 == 0 and len(agent.memory) >= agent.bs:
                agent.learn()
                agent.soft_update(agent.tau)

            state = next_state.copy()
            episodic_reward += reward

        rewards_log.append(episodic_reward)
        average_log.append(np.mean(rewards_log[-100:]))
        print('\rEpisode {}, Reward {:.3f}, Average Reward {:.3f}'.format(i, episodic_reward, average_log[-1]), end='')
        if i % 100 == 0:
            print()

        eps = max(eps * eps_decay, eps_min)

    return rewards_log

In [ ]:
train(env, agent, num_episode, eps, eps_decay, eps_min, max_t, num_frame=1, constant=C)

Episode 100, Reward -81685.373, Average Reward -221903.016
Episode 200, Reward -124020.049, Average Reward -142613.512
Episode 300, Reward -140494.203, Average Reward -96832.126
Episode 400, Reward -175470.435, Average Reward -57965.093
Episode 500, Reward -53623.772, Average Reward -33116.166
Episode 600, Reward -8090.620, Average Reward -16072.948
Episode 700, Reward 11045.635, Average Reward -6079.393
Episode 800, Reward -5640.702, Average Reward -2564.333
Episode 900, Reward 9757.885, Average Reward 4216.378
Episode 1000, Reward -13335.116, Average Reward 7197.827
Episode 1100, Reward 8898.823, Average Reward 3325.287
Episode 1200, Reward 18774.729, Average Reward 5363.876
Episode 1300, Reward 24773.072, Average Reward 7486.154
Episode 1400, Reward 14051.282, Average Reward 5249.853
Episode 1500, Reward 1598.757, Average Reward 5672.483
Episode 1600, Reward -12475.325, Average Reward 6987.762
Episode 1700, Reward 8040.067, Average Reward 3507.255
Episode 1800, Reward 10616.257, Ave